In [2]:
import nltk
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tokenize import word_tokenize
# nltk.download('twitter_samples')

In [24]:
# Normalize the words
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Lenovo IdeaPad
[nltk_data]     JQ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Lenovo IdeaPad
[nltk_data]     JQ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo IdeaPad
[nltk_data]     JQ\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to C:\Users\Lenovo IdeaPad
[nltk_data]     JQ\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [4]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [5]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [6]:
stop_words = stopwords.words('english')

In [7]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [8]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
freq_dist_pos = FreqDist(all_pos_words)

In [9]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [10]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [14]:
dataset

[({'go': True,
   'please': True,
   'click': True,
   'link': True,
   'checkout': True,
   ':)': True},
  'Positive'),
 ({'always': True,
   'part': True,
   'defenitely': True,
   '...': True,
   '♬': True,
   ':p': True},
  'Positive'),
 ({'feel': True, 'leave': True, ':(': True}, 'Negative'),
 ({"here's": True,
   'fan': True,
   'make': True,
   'art': True,
   'u': True,
   ':)': True,
   'guy': True,
   'phone': True,
   'lew': True,
   'bend': True,
   ':d': True},
  'Positive'),
 ({'thanks': True, 'support': True, ':)': True}, 'Positive'),
 ({'get': True,
   'well': True,
   'soon': True,
   'dan': True,
   'big': True,
   'hug': True,
   ':)': True},
  'Positive'),
 ({'also': True, 'gl': True, "tonight's": True, 'show': True, ':d': True},
  'Positive'),
 ({'like': True, 'moment': True, '...': True, ':-(': True}, 'Negative'),
 ({'inter': True,
   'offer': True,
   'nemanja': True,
   'vidic': True,
   'roma': True,
   '...': True,
   'want': True,
   'either': True,
   ':(': 

In [12]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

In [13]:
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2071.9 : 1.0
                      :) = True           Positi : Negati =    986.9 : 1.0
                 welcome = True           Positi : Negati =     35.6 : 1.0
                     sad = True           Negati : Positi =     31.5 : 1.0
                follower = True           Positi : Negati =     20.6 : 1.0
                    glad = True           Positi : Negati =     19.0 : 1.0
                     bam = True           Positi : Negati =     17.6 : 1.0
                     x15 = True           Negati : Positi =     15.0 : 1.0
                   enjoy = True           Positi : Negati =     13.2 : 1.0
                   great = True           Positi : Negati =     12.9 : 1.0
None


In [55]:
custom_tweet = 'I’d love to see Trump day any of this shit he’s saying about Obama to his face. There’s no way he’d have the labs'
custom_tokens = remove_noise(word_tokenize(custom_tweet))
print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


In [56]:
import pickle
f = open('my_classifier.pickle', 'wb')
pickle.dump(classifier, f)
f.close()